In [ ]:
# IDEA: use ProtTrans pipeline as feature extraction (instead of embeddings) and build model on top of it
# check whether input format makes sense
# https://github.com/agemagician/ProtTrans/blob/master/Embedding/PyTorch/Basic/ProtBert.ipynb

In [1]:
import os
import sys
import csv
import pickle
import random
import numpy as np
from time import time
from tqdm import tqdm

from typing import List, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torchtext.vocab import build_vocab_from_iterator

In [2]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(1234)

In [3]:
# def read_data(path):
#     with open(path, 'r') as csvfile:
#         train_data = list(csv.reader(csvfile))[1:] # skip col name
#         sents, lbls = [], []
#         for i in range(0, len(train_data), 16):
#             s, l = zip(*train_data[i:i+16])
#             sents.append(s)
#             lbls.append(l)
#     return sents, lbls

def read_data(path):
    with open(path, 'r') as csvfile:
        train_data = list(csv.reader(csvfile))[1:] # skip col name
        sents, lbls = [], []
        for s, l in train_data:
            sents.append(s)
            lbls.append(l)
    return sents, lbls

# number of trainable parameters in model
def get_total_model_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
# class CleavageDataset(Dataset):
#     def __init__(self, seq, lbl):
#         self.seq = seq
#         self.lbl = lbl
    
#     def __getitem__(self, idx):
#         return self.seq[idx], self.lbl[idx]
    
#     def __len__(self):
#         return len(self.lbl)
    
# class CleavageBatch:
#     def __init__(self, batch: List[Tuple[str, str]]):
#         ordered_batch = list(zip(*batch))
#         self.seq = torch.tensor([encode_text(seq) for seq in ordered_batch[0]], dtype=torch.int64)
#         self.lbl = torch.tensor([int(l) for l in ordered_batch[1]], dtype=torch.float)
        
#     def pin_memory(self):
#         self.seq = self.seq.pin_memory()
#         self.lbl = self.lbl.pin_memory()
#         return self
    
# def collate_wrapper(batch):
#     return CleavageBatch(batch)

In [4]:
class CleavageDataset(Dataset):
    def __init__(self, seq, lbl):
        self.seq = seq
        self.lbl = lbl
    
    def __getitem__(self, idx):
        return self.seq[idx], self.lbl[idx]
    
    def __len__(self):
        return len(self.lbl)
    
def collate_batch(batch):
    ordered_batch = list(zip(*batch))
    seq = torch.tensor([encode_text(seq) for seq in ordered_batch[0]], dtype=torch.int64)
    lbl = torch.tensor([int(l) for l in ordered_batch[1]], dtype=torch.float)
    return seq.to(device), lbl.to(device)

In [5]:
class QuadBiLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, rnn_size1, rnn_size2, rnn_size3, rnn_size4, hidden_size, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=embedding_dim,
        )
        
        self.dropout=nn.Dropout(dropout)
        
        self.lstm1 = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=rnn_size1,
            bidirectional=True,
            batch_first=True
        )
        
        self.lstm2 = nn.LSTM(
            input_size=2*rnn_size1,
            hidden_size=rnn_size2,
            bidirectional=True,
            batch_first=True
        )
        
        self.lstm3 = nn.LSTM(
            input_size=2*rnn_size2,
            hidden_size=rnn_size3,
            bidirectional=True,
            batch_first=True
        )
        
        self.lstm4 = nn.LSTM(
            input_size=2*rnn_size3,
            hidden_size=rnn_size4,
            bidirectional=True,
            batch_first=True
        )
        
        self.fc1 = nn.Linear(rnn_size4 * 2, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)
        
    def forward(self, seq):
        # input shape: (batch_size, seq_len=10)
        embedded = self.dropout(self.embedding(seq))
        
        # input shape: (batch_size, seq_len, embedding_dim)
        out, _ = self.lstm1(embedded)
        
        # input shape: (batch_size, seq_len, 2*hidden_size)
        out, _ = self.lstm2(out)
        
        # input shape: (batch_size, seq_len, 2*hidden_size)
        out, _ = self.lstm3(out)
        
        # input shape: (batch_size, seq_len, 2*hidden_size)
        out, _ = self.lstm4(out)
        
        # input shape: (batch_size, seq_len, 2*hidden_size)
        pooled = torch.mean(out, dim=1)
        
        # input shape; (batch_size, 2*hidden_size)
        out = self.dropout(F.relu(self.fc1(pooled)))
        
        # input shape: (batch_size, hidden_size)
        out = self.fc2(out).squeeze()
        return out # shape: (batch_size)

In [ ]:
# def process(model, loader, criterion, optim=None):
#     epoch_loss, num_correct, total = 0, 0, 0
    
#     for batch in tqdm(
#         loader,
#         desc="Train: " if optim is not None else "Eval: ",
#         file=sys.stdout,
#         unit="batches"
#     ):
#         seq, lbl = batch.seq, batch.lbl
#         seq, lbl = seq.to(device), lbl.to(device)
        
#         scores = model(seq)
#         loss = criterion(scores, lbl)
        
#         if optim is not None:
#             optim.zero_grad()
#             loss.backward()
#             optim.step()
        
#         epoch_loss += loss.item()
#         num_correct += ((scores > 0) == lbl).sum()
#         total += len(seq)
#     return epoch_loss / total, num_correct / total

In [6]:
def process(model, loader, criterion, optim=None):
    epoch_loss, num_correct, total = 0, 0, 0
    
    for seq, lbl in loader:
        scores = model(seq)
        loss = criterion(scores, lbl)
        
        if optim is not None:
            optim.zero_grad()
            loss.backward()
            optim.step()
        
        epoch_loss += loss.item()
        num_correct += ((scores > 0) == lbl).sum()
        total += len(seq)
    return epoch_loss / total, num_correct / total

In [7]:
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# load train and dev data
train_seqs, train_lbl = read_data('../data/n_train.csv')
dev_seqs, dev_lbl = read_data('../data/n_val.csv')

# create vocab from train seqs
vocab = build_vocab_from_iterator(train_seqs, specials=['<UNK>'])
vocab.set_default_index(vocab['<UNK>'])
encode_text = lambda x: vocab(list(x))

In [ ]:
NUM_EPOCHS = 2
BATCH_SIZE = 512
VOCAB_SIZE = len(vocab)
EMBEDDING_DIM = 128
RNN_SIZE1 = 128
RNN_SIZE2 = 512
RNN_SIZE3 = 256
RNN_SIZE4 = 128
HIDDEN_SIZE = 128
DROPOUT = 0.5
LEARNING_RATE = 1e-4

model = QuadBiLSTM(
    vocab_size=VOCAB_SIZE,
    embedding_dim=EMBEDDING_DIM,
    rnn_size1=RNN_SIZE1,
    rnn_size2=RNN_SIZE2,
    rnn_size3=RNN_SIZE3,
    rnn_size4=RNN_SIZE4,
    hidden_size=HIDDEN_SIZE,
    dropout=DROPOUT
).to(device)

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.BCEWithLogitsLoss()

# create train and dev loader
train_data = CleavageDataset(train_seqs, train_lbl)
train_loader = DataLoader(train_data, batch_size = BATCH_SIZE, shuffle=True, collate_fn=collate_wrapper, pin_memory=True, num_workers=10)

dev_data = CleavageDataset(dev_seqs, dev_lbl)
dev_loader = DataLoader(dev_data, batch_size = BATCH_SIZE, shuffle=True, collate_fn=collate_wrapper, pin_memory=True, num_workers=10)

print(f"Total trainable model parameters: {get_total_model_params(model):,}")

In [ ]:
start = time()
print("Starting Training.")
highest_val_acc = 0
train_losses, train_accuracies= [], []
val_losses, val_accuracies = [], []

for epoch in range(1, NUM_EPOCHS + 1):
    model.train()
    train_loss, train_acc = process(model, train_loader, criterion, optimizer)
    
    model.eval()
    with torch.no_grad():
        val_loss, val_acc = process(model, dev_loader, criterion)
        
    # save current acc, loss
    train_losses.append((epoch, train_loss))
    train_accuracies.append((epoch, train_acc))
    val_losses.append((epoch, val_loss))
    val_accuracies.append((epoch, val_acc))
    
    if val_acc > highest_val_acc:
        highest_val_acc = val_acc
        path = f"../params/n_term/quadBiLSTM/acc{val_acc:.4f}_epoch{epoch}.pt"
        torch.save(model.state_dict(), path)
        
    print(
        f"Training:   [Epoch {epoch:2d}, Loss: {train_loss:8.4f}, Acc: {train_acc:.4f}]"
    )
    print(f"Evaluation: [Epoch {epoch:2d}, Loss: {val_loss:8.4f}, Acc: {val_acc:.4f}]")
    
print("Finished Training.")
train_time = (time() - start) / 60
print(f"Training took {train_time} minutes.")

In [ ]:
# save training stats
lsts = [train_losses, train_accuracies, val_losses, val_accuracies, train_time]
names = [
    "train_losses",
    "train_accuracies",
    "val_losses",
    "val_accuracies",
    "train_time",
]
to_save = dict()
for name, lst in zip(names, lsts):
    to_save[name] = lst

with open(f"../params/n_term/quadBiLSTM/metrics.pkl", "wb") as f:
    pickle.dump(to_save, f, pickle.HIGHEST_PROTOCOL)

print("Finished Saving Details.")

In [ ]:
# implement ray tune
# implement forward-backward representations

In [8]:
from ray import tune
from ray.tune.schedulers import ASHAScheduler

In [15]:
def train(config):
    # create train and dev loader
    train_data = CleavageDataset(train_seqs, train_lbl)
    train_loader = DataLoader(train_data, batch_size = 512, shuffle=True, collate_fn=collate_batch)

    dev_data = CleavageDataset(dev_seqs, dev_lbl)
    dev_loader = DataLoader(dev_data, batch_size = 512, shuffle=True, collate_fn=collate_batch)
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    model = QuadBiLSTM(
        vocab_size=VOCAB_SIZE,
        embedding_dim=config['embedding_dim'],
        rnn_size1=config['rnn_size1'],
        rnn_size2=config['rnn_size2'],
        rnn_size3=config['rnn_size3'],
        rnn_size4=config['rnn_size4'],
        hidden_size=config['hidden_size'],
        dropout=config['dropout']
    ).to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=config['lr'])
    criterion = nn.BCEWithLogitsLoss()
    
    # normal train loop
    for epoch in range(1, NUM_EPOCHS + 1):
        model.train()
        train_loss, train_acc = process(model, train_loader, criterion, optimizer)

        model.eval()
        with torch.no_grad():
            val_loss, val_acc = process(model, dev_loader, criterion)
            
        tune.report(mean_accuracy=val_acc)

In [16]:
search_space = {
    'embedding_dim': tune.choice([128]),
    'rnn_size1': tune.choice([128]),
    'rnn_size2': tune.choice([512]),
    'rnn_size3': tune.choice([256]),
    'rnn_size4': tune.choice([128]),
    'hidden_size': tune.choice([128]),
    'dropout': tune.choice([0.5]),
    'lr': tune.qloguniform(1e-4, 1e-1, 5e-5),
}

In [17]:
analysis = tune.run(
    train,
    num_samples=20,
    scheduler=ASHAScheduler(metric='mean_accuracy', mode='max'),
    config=search_space,
    resources_per_trial={
        'gpu': 1,
        'cpu': 20
    }
)

2022-08-06 13:54:03,899	WARNING worker.py:1404 -- Warning: The actor ImplicitFunc is very large (22 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.


Trial name,status,loc,dropout,embedding_dim,hidden_size,lr,rnn_size1,rnn_size2,rnn_size3,rnn_size4
train_75fca_00000,RUNNING,192.168.178.26:20036,0.5,128,128,0.00205,128,512,256,128
train_75fca_00001,PENDING,,0.5,128,128,0.05685,128,512,256,128
train_75fca_00002,PENDING,,0.5,128,128,0.0131,128,512,256,128
train_75fca_00003,PENDING,,0.5,128,128,0.0173,128,512,256,128
train_75fca_00004,PENDING,,0.5,128,128,0.00795,128,512,256,128
train_75fca_00005,PENDING,,0.5,128,128,0.0003,128,512,256,128
train_75fca_00006,PENDING,,0.5,128,128,0.00385,128,512,256,128
train_75fca_00007,PENDING,,0.5,128,128,0.00275,128,512,256,128
train_75fca_00008,PENDING,,0.5,128,128,0.0041,128,512,256,128
train_75fca_00009,PENDING,,0.5,128,128,0.00015,128,512,256,128


(train pid=20036) 2022-08-06 13:54:05,155	ERROR function_runner.py:286 -- Runner Thread raised error.
(train pid=20036) Traceback (most recent call last):
(train pid=20036)   File "/home/ingo/miniconda3/envs/aaml/lib/python3.10/site-packages/ray/tune/function_runner.py", line 277, in run
(train pid=20036)     self._entrypoint()
(train pid=20036)   File "/home/ingo/miniconda3/envs/aaml/lib/python3.10/site-packages/ray/tune/function_runner.py", line 349, in entrypoint
(train pid=20036)     return self._trainable_func(
(train pid=20036)   File "/home/ingo/miniconda3/envs/aaml/lib/python3.10/site-packages/ray/util/tracing/tracing_helper.py", line 462, in _resume_span
(train pid=20036)     return method(self, *_args, **_kwargs)
(train pid=20036)   File "/home/ingo/miniconda3/envs/aaml/lib/python3.10/site-packages/ray/tune/function_runner.py", line 645, in _trainable_func
(train pid=20036)     output = fn()
(train pid=20036)   File "/tmp/ipykernel_18009/174204523.py", line 12, in train
(trai

Result for train_75fca_00000:
  date: 2022-08-06_13-54-05
  experiment_id: b7556e5ea9b5423a81300240b3565cec
  hostname: arch
  node_ip: 192.168.178.26
  pid: 20036
  timestamp: 1659786845
  trial_id: 75fca_00000
  


(train pid=20094) 2022-08-06 13:54:06,936	ERROR function_runner.py:286 -- Runner Thread raised error.
(train pid=20094) Traceback (most recent call last):
(train pid=20094)   File "/home/ingo/miniconda3/envs/aaml/lib/python3.10/site-packages/ray/tune/function_runner.py", line 277, in run
(train pid=20094)     self._entrypoint()
(train pid=20094)   File "/home/ingo/miniconda3/envs/aaml/lib/python3.10/site-packages/ray/tune/function_runner.py", line 349, in entrypoint
(train pid=20094)     return self._trainable_func(
(train pid=20094)   File "/home/ingo/miniconda3/envs/aaml/lib/python3.10/site-packages/ray/util/tracing/tracing_helper.py", line 462, in _resume_span
(train pid=20094)     return method(self, *_args, **_kwargs)
(train pid=20094)   File "/home/ingo/miniconda3/envs/aaml/lib/python3.10/site-packages/ray/tune/function_runner.py", line 645, in _trainable_func
(train pid=20094)     output = fn()
(train pid=20094)   File "/tmp/ipykernel_18009/174204523.py", line 12, in train
(trai

...

Trial name,status,loc,dropout,embedding_dim,hidden_size,lr,rnn_size1,rnn_size2,rnn_size3,rnn_size4
train_75fca_00006,PENDING,,0.5,128,128,0.00385,128,512,256,128
train_75fca_00007,PENDING,,0.5,128,128,0.00275,128,512,256,128
train_75fca_00008,PENDING,,0.5,128,128,0.0041,128,512,256,128
train_75fca_00009,PENDING,,0.5,128,128,0.00015,128,512,256,128
train_75fca_00010,PENDING,,0.5,128,128,0.02,128,512,256,128
train_75fca_00011,PENDING,,0.5,128,128,0.0246,128,512,256,128
train_75fca_00012,PENDING,,0.5,128,128,0.079,128,512,256,128
train_75fca_00013,PENDING,,0.5,128,128,0.0001,128,512,256,128
train_75fca_00014,PENDING,,0.5,128,128,0.0002,128,512,256,128
train_75fca_00015,PENDING,,0.5,128,128,0.00095,128,512,256,128


2022-08-06 13:54:16,716	WARNING tune.py:682 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
(train pid=20288) 2022-08-06 13:54:16,958	ERROR function_runner.py:286 -- Runner Thread raised error.
(train pid=20288) Traceback (most recent call last):
(train pid=20288)   File "/home/ingo/miniconda3/envs/aaml/lib/python3.10/site-packages/ray/tune/function_runner.py", line 277, in run
(train pid=20288)     self._entrypoint()
(train pid=20288)   File "/home/ingo/miniconda3/envs/aaml/lib/python3.10/site-packages/ray/tune/function_runner.py", line 349, in entrypoint
(train pid=20288)     return self._trainable_func(
(train pid=20288)   File "/home/ingo/miniconda3/envs/aaml/lib/python3.10/site-packages/ray/util/tracing/tracing_helper.py", line 462, in _resume_span
(train pid=20288)     return method(self, *_args, **_kwargs)
(train pid=20288)   File "/home/

Result for train_75fca_00006:
  date: 2022-08-06_13-54-16
  experiment_id: 303a610da3564073b29ebf040a7d61a5
  hostname: arch
  node_ip: 192.168.178.26
  pid: 20288
  timestamp: 1659786856
  trial_id: 75fca_00006
  


Trial name,status,loc,dropout,embedding_dim,hidden_size,lr,rnn_size1,rnn_size2,rnn_size3,rnn_size4
train_75fca_00007,PENDING,,0.5,128,128,0.00275,128,512,256,128
train_75fca_00008,PENDING,,0.5,128,128,0.0041,128,512,256,128
train_75fca_00009,PENDING,,0.5,128,128,0.00015,128,512,256,128
train_75fca_00010,PENDING,,0.5,128,128,0.02,128,512,256,128
train_75fca_00011,PENDING,,0.5,128,128,0.0246,128,512,256,128
train_75fca_00012,PENDING,,0.5,128,128,0.079,128,512,256,128
train_75fca_00013,PENDING,,0.5,128,128,0.0001,128,512,256,128
train_75fca_00014,PENDING,,0.5,128,128,0.0002,128,512,256,128
train_75fca_00015,PENDING,,0.5,128,128,0.00095,128,512,256,128
train_75fca_00016,PENDING,,0.5,128,128,0.00235,128,512,256,128


2022-08-06 13:54:17,383	ERROR tune.py:743 -- Trials did not complete: [train_75fca_00000, train_75fca_00001, train_75fca_00002, train_75fca_00003, train_75fca_00004, train_75fca_00005, train_75fca_00006, train_75fca_00007, train_75fca_00008, train_75fca_00009, train_75fca_00010, train_75fca_00011, train_75fca_00012, train_75fca_00013, train_75fca_00014, train_75fca_00015, train_75fca_00016, train_75fca_00017, train_75fca_00018, train_75fca_00019]
2022-08-06 13:54:17,384	INFO tune.py:747 -- Total run time: 16.85 seconds (16.38 seconds for the tuning loop).
2022-08-06 13:54:17,384	WARNING tune.py:753 -- Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`


In [ ]:
dfs = analysis.trial_dataframes